### 트윗 스트리밍  
twitter API의 한계로 7일 이전의 트윗은 불러올 수 없다. 그러나 다른 방법(twitterscraper 등의 외부 오픈 패키지)는 모두 막혀 진행이 어려워 소규모지만 데이터를 모아보려 한다.

In [5]:
import twitter #twitter와 python의 연동을 위해서
import json #json 다루기 위해서
import re #정규표현식을 다루기 위해

In [1]:
#API를 사용하기 위한 키
twitter_consumer_key = "YOUR KEY"
twitter_consumer_secret = "YOUR KEY"
twitter_access_token = "YOUR KEY"
twitter_access_secret = "YOUR KEY"

twitter_api = twitter.Api(consumer_key=twitter_consumer_key,
                          consumer_secret=twitter_consumer_secret, 
                          access_token_key=twitter_access_token, 
                          access_token_secret=twitter_access_secret)

NameError: name 'twitter' is not defined

찾은 모든 방법에서 쿼리가 없는 경우가 없어서(즉, 실시간으로 모든 트윗 혹은 공백이 포함된 트윗을 스크래핑할 수가  없어서), 현시점 가장 트윗이 많이 발생할 것이라 생각하는 '올림픽'을 검색해 스트리밍 방식으로 트위터를 모았다. 약 6시간동안 모았으며, 스트리밍한 json 형태의 내용을 txt 문서로 저장하였다. 보관을 위해서 저장한 파일은 raw_data.zip으로 압축하였다.

In [10]:
query = ["올림픽"]
output_file_name = "raw_data_twit.txt"
with open(output_file_name, "w", encoding="utf-8") as output_file:
    stream = twitter_api.GetStreamFilter(track=query)
    while True:
        for tweets in stream:
            tweet = json.dumps(tweets, ensure_ascii=False)
            print(tweet, file=output_file, flush=True)

KeyboardInterrupt: 

중단 기능을 만들지 않아서 Error가 발생한 것처럼 보이지만, 실제로 파일을 확인해보면 약 6시간동안 약 15,000개의 트윗을 저장한 것을 볼 수 있다. 이를 바탕으로 필요없는 것들을 기초적으로 전처리하였다.

In [2]:
#차후 ainize를 활용해 KOGPT를 학습시키기 위해 전처리 후 토큰화 하기 전의 저장할 파일 생성
file = open('edited_twit.txt', 'w')

In [6]:
#파일 읽기 모드로 열기
f = open("raw_data_twit.txt", 'r', encoding='UTF8')
#한줄씩 불러와서 전처리하기
for lines in f:
    line = json.loads(lines) #json 형태로 불러와서
    line = line['text'] #텍스트 파트만 정해놓고
    line = re.sub('https?://\S+', '', line) #링크 등에 쓰인 https 주소들 제거
    line = re.sub('@\S+', '', line) # @아이디 인 멘션 제거
    line = re.sub('RT', '', line) #RT 표시 제거, 다른 RT 글자가 제거될 수도 있지만 일단 제거
    line = re.sub('[0-9a-zA-Z]', '', line) #숫자와 영어 제거
    line = re.sub('\n', '', line) #엔터 제거
    line = re.sub(r'[?~!@#:+$"%^&,*()_=-]', '', line) #기본적인 특수문자 제거 1
    line = re.sub(r"[']", '', line) #기본적인 특수문자 제거 2
    line = re.sub("[.]{2}", '▒', line) #...은 남기기 위해 임시로 ▒로 치환
    line = re.sub("[.]", ' ', line) #하나만 있는 건 제거
    line = re.sub("▒", '...', line) #다시 말줄임표로 전환
    file.write(str(line) + '\n') #한줄씩 쓴다

In [8]:
#파일 닫기
file.close()

약 15,000줄의 데이터를 확보함

##### *트윗의 전처리에 대해서  
숫자(ex: 2022 올림픽, 1000m) 등은 상기한 대로의 쓰임이 많아 제거하였다. 영어의 경우, 종종 쓰이지만 우선 다 제거하였다. 특수문자의 경우, ... 은 어투를 구성한다 생각해 제거하지 않았고, 🤩과 같은 이모지, ㅠㅠㅠ 혹은 ㅋㅋㅋ 등 역시 감정에 영향을 끼친다 생각해 제거하지 않았다.